# Clean-up Memory

The reference implementation of VSA lisp has a simple clean-up memory which 
acts up when used as a drop-in replacement with FHRRs. Therefore, we will 
here be inspecting the dynamics of the simple clean-up memory in order
to diagnose the problem.

In [129]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from vsa.fhrr import FHRR
dim = 600
np.random.seed(0)

fhrr = FHRR(dim=dim)

In [167]:
class SimpleCleanup:
    mem: np.ndarray
    """The memory matrix."""
    dim: int
    """Dimensionality of the vectors involved"""
    cap: int
    """Current storage capacity."""
    incr: int
    """Increment to increase capacity `cap` with."""
    size: int
    """Current count of traces stored."""

    def __init__(self, dim: int, cap: int = 100) -> None:
        self.dim = dim
        self.cap = cap
        self.incr = cap
        self.size = 0
        self.mem = np.zeros(shape=(self.cap, self.dim), dtype=complex)

    def memorize(self, v: np.ndarray) -> np.ndarray:
        if self.size >= self.cap:
            self.mem = np.concatenate(
                [self.mem, np.zeros((self.incr, self.dim))], axis=0
            )
            self.cap += self.incr

        self.mem[self.size, :] = v
        self.size += 1
        return v

    def recall(self, v: np.ndarray) -> np.ndarray:
        #act = np.vectorize(lambda v, m: fhrr.sim(m, v), signature='(n),(n,m)->(m)')
        activations = [fhrr.sim(v, m) for m in self.mem]
        return self.mem[np.argmax(activations), :]

# First problems

Immediately, this definition has some problems. 

In [173]:
for _ in range(10):
    mem = SimpleCleanup(dim)
    fhrr.set_symbols(["red", "blue", "green", "yellow", "white", "cyan", "magenta"])

    for _, value in fhrr.symbols.items():
        mem.memorize(value)

    print(fhrr.sim(mem.recall(fhrr.superpose(fhrr["red"], fhrr["blue"])), fhrr["blue"]))

0.06879334063865698
-0.05061550459620943
1.0
-0.06072032028344405
1.0
0.01988138630204328
0.002316238090864585
1.0
-0.03611916054401311
1.0


In [157]:
test = np.array([[1,2,3], [4,5,6]])
for x in test:
    print(x)

[1 2 3]
[4 5 6]
